In [ ]:
# score per person per paragraph
import os
import time
from collections import defaultdict

import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import json

import utils
from utils import extract_score

label_to_score = utils.label_to_score

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [ ]:
speaker_transcripts = pd.read_csv('../data/transcripts_byspeaker.csv')

In [ ]:
prompt_template = """
<sentences>
INPUT
</sentences>
<labels>
Dovish: Strongly expresses a belief that the economy may be
growing too slowly and may need stimulus through mon-
etary policy.
Mostly dovish: Overall message expresses a belief that the economy may
be growing too slowly and may need stimulus through
monetary policy.
Neutral: Expresses neither a hawkish nor dovish view and is
mostly objective.
Mostly hawkish: Overall message expresses a belief that the economy is
growing too quickly and may need to be slowed down
through monetary policy.
Hawkish: Strongly expresses a belief that the economy is growing
too quickly and may need to be slowed down through monetary policy.
</labels>
Are sentences are all phrases said by a single speaker in a transcript. Which label best applies applies all of the sentences taken as a whole (Dovish, Mostly Dovish, Neutral, Mostly Hawkish, Hawkish)?
"""

In [ ]:
from collections import defaultdict
import ast

scores = defaultdict(lambda: defaultdict(list))

In [ ]:

for i, transcript in tqdm(speaker_transcripts.iterrows()):
    transcript_scores = scores[transcript["date"]]
    speakers_dict = ast.literal_eval(transcript["speaker_text"])
    for speaker in speakers_dict:
        speaker_scores = transcript_scores[speaker]
        speaker_statements = "".join(speakers_dict[speaker])
        # print(speaker_statements)
        response = None
        while response is None:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-4-32k",
                    messages=[
                            {"role": "user", "content": prompt_template.replace("INPUT", speaker_statements)},
                        ],
                    temperature=0
                )
            except Exception as e:
                print(e)
                time.sleep(1)
        score = extract_score(response["choices"][0]["message"]["content"])
        speaker_scores.append(score)


In [ ]:
import json
with open('../results/transcripts_full_score.json', 'w') as f:
    json.dump(scores, f, indent=4)